In [1]:
test = '''
contract sum(n : int){

    let x: int ;

    entry sum(n: int){
        let a: int = 2;
        let b: int = 3;
        while (a < b) {
            x = a + b;
            a = 5;
        }
        
        
    }
}
'''



In [2]:
from visitors.string_rep_visitor import FormatVisitor
from utils import run_tzscript_ast_building_pipeline

ast = run_tzscript_ast_building_pipeline(test)
format = FormatVisitor()
print(format.visit(ast))

0 	 <program> -> .contractid(<op-param-list{<stat_program_list>}, {'$'}
	 S' -> .<program>, {'$'} 

1 	 <program> -> contract.id(<op-param-list{<stat_program_list>}, {'$'} 

2 	 <program> -> contractid.(<op-param-list{<stat_program_list>}, {'$'} 

3 	 <param-list> -> .<param>, {',', ')'}
	 <program> -> contractid(.<op-param-list{<stat_program_list>}, {'$'}
	 <op-param-list -> .<param-list>), {'{'}
	 <op-param-list -> .), {'{'}
	 <param-list> -> .<param-list>,<param>, {',', ')'}
	 <param> -> .id:type, {',', ')'} 

4 	 <param-list> -> <param>., {',', ')'} 

5 	 <program> -> contractid(<op-param-list.{<stat_program_list>}, {'$'} 

6 	 <program> -> contractid(<op-param-list{.<stat_program_list>}, {'$'}
	 <stat_program_list> -> .<stat_program_list><stat_program>, {'func', '}', 'entry', 'let'}
	 <def-func> -> .funcid(<param-list>):type{<stat_list>}, {'func', '}', 'entry', 'let'}
	 <stat_program_list> -> .<stat_program>, {'func', '}', 'entry', 'let'}
	 <stat_program> -> .<def-entry>, {'let', 

In [3]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
from visitors.hl_string_repre import HLReprVisitor

hl = TzScriptToHighLevelIrVisitor()
hl_ir = hl.visit(ast)


num
num
num


In [4]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
high_level_ir = TzScriptToHighLevelIrVisitor()
ir = high_level_ir.visit(ast)

num
num
num


In [5]:
from visitors.hl_string_repre import HLReprVisitor
hl_repr = HLReprVisitor()
print(hl_repr.visit(ir))

contract { entrypoint{...} storage{...} code{...}} 
	entrypoint  { <id,params> , <id,params> , ... , <id,params> }
		\__EntryPointDeclarationNode: sum (<param> ... <param>)
			\__AttrDeclarationNode: n : int
	storage { <id,type> , <id,type> , ... , <id,type> }
		\__StorageDeclarationNode: x : int
	code { <stat> , <stat> , ... , <stat> }
		\__IfEntryNode: if sum then 
			\__PushValueNode: push 2 num
			\__PushVariableNode: push a int
			\__PushValueNode: push 3 num
			\__PushVariableNode: push b int
			while <expr> loop { <stat> , <stat> , ... , <stat> }
				\__GetToTopNode: get a
				\__GetToTopNode: get b
				\__ LessThanNode
					\__GetToTopNode: get a
					\__GetToTopNode: get b
					\__ PlusNode
					\__ReplaceVariableNode: x
					\__PushValueNode: push 5 num
					\__ReplaceVariableNode: a


In [6]:
from visitors.michelson_generator_visitor import MichelsonGenerator
visit_generator = MichelsonGenerator()
code = visit_generator.visit(ir)

final code: 
parameter <parser.tzscript_types.TzScriptInt object at 0x105e83ad0> %sum;
storage <parser.tzscript_types.TzScriptInt object at 0x105e83ad0>;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
SWAP;
SWAP;
DUP;
DIG 2;
SWAP;
SWAP;
SUB;
LT;
LOOP {DIG 4;
DIG 2;
DUP;
DIG 2;
SWAP;
ADD;
DIG 4;
DROP;
PUSH nat 5;
DIG 4;
DROP;
DIG 3;
DUP;
DIG 2;
SWAP;
SWAP;
SUB;
LT;
IF
{ PUSH False; }
{ PUSH True; }
} 
DIG 0;
DROP;
DIG 0;
DROP;
DIG 0;
DROP;

